<a href="https://colab.research.google.com/github/abhilashhn1993/movie-recommendation-engine/blob/main/Movie_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing the dependencies

In [ ]:
#Java JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Downloading Spark
!wget -q http://apache.mirrors.pair.com/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz 
#Unzipping the hadoop file
!tar -xvf spark-3.0.1-bin-hadoop3.2.tgz

In [4]:
#Fetching the MovieLens dataset
!wget http://files.grouplens.org/datasets/movielens/ml-latest.zip

--2020-11-30 23:23:51--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 264.28M  76.3MB/s    in 3.7s    

2020-11-30 23:23:55 (71.8 MB/s) - ‘ml-latest.zip’ saved [277113433/277113433]



In [5]:
#Unzip the file
!unzip ml-latest.zip

Archive:  ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [59]:
###################### SPARK SETUP ################################
#Install findspark
!pip install -q findspark

In [60]:
#Setting up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [61]:
#Initialize Spark session using findspark lib
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Read the data file

In [62]:
#get the ratings.csv file
ratings ='/content/ml-latest/ratings.csv'
movies = '/content/ml-latest/movies.csv'
links = '/content/ml-latest/links.csv'

In [63]:
#Read the data files
ratings = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load(ratings,header=True)
              

In [64]:
movies = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load(movies,header=True)

In [65]:
links = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load(links,header=True)

In [28]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [40]:
movie_df = spark.read.csv("/content/ml-latest/movies.csv", inferSchema=True, header=True)

In [25]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [9]:
#print the schema 
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [22]:
print('No. of rows: %d' % ratings.count())
ratings.show(5)

No. of row: 27753444
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [29]:
#Check the top 20 rows of the data
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
|     1|   1590|   2.5|1256677236|
|     1|   1591|   1.5|1256677475|
|     1|   2134|   4.5|1256677464|
|     1|   2478|   4.0|1256677239|
|     1|   2840|   3.0|1256677500|
|     1|   2986|   2.5|1256677496|
|     1|   3020|   4.0|1256677260|
|     1|   3424|   4.5|1256677444|
|     1|   3698|   3.5|1256677243|
|     1|   3826|   2.0|1256677210|
|     1|   3893|   3.5|1256677486|
|     2|    170|   3.5|1192913581|
|     2|    849|   3.5|1192913537|
|     2|   1186|   3.5|1192913611|
|     2|   1235|   3.0|1192913585|
+------+-------+------+----------+
only showing top 20 rows



In [11]:
#Removing timestamp column, so dropping it
data2 = ratings.drop("timestamp")

#print the schema now and check that timestamp column is dropped
data2.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [13]:
# Randomly split the data into train and test where 80% data is in train and remaining is test
train, test = data2.randomSplit([0.8, 0.2])

In [14]:
# Build a recommendation model using Alternating Least Squares
# Evaluate the model by computing the RMSE on the test data
model = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True, coldStartStrategy="drop").fit(train)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)


# print the RMSE
print("New RMSE: ", evaluator.evaluate(model.transform(test)))

New RMSE:  0.8252457183252592


In [15]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

# Now we try to improve the performance of the original model using cross validation and solve the cold-start problem.
# we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

model = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, coldStartStrategy="drop")

# We use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder() \
    .addGrid(model.regParam, [0.1, 0.01, 0.001, 0.0001]) \
    .addGrid(model.rank, [5, 10, 15, 20]) \
    .build()

# Define a cross-validator object
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
crossvalidation = CrossValidator(estimator = model,
                     estimatorParamMaps = paramGrid,
                     evaluator = evaluator,
                     numFolds=5)

In [16]:
# Run cross-validation, and choose the best set of parameters.
Best_model = crossvalidation.fit(train).bestModel

Printing the Best Model's parameter values

In [73]:
#Print best_model
print(type(Best_model))

#Complete the code below to extract the ALS model parameters
print("**Best Model**")

#Print "Rank"
print("  Rank:", Best_model._java_obj.parent().getRank())

#Print "MaxIter"
print("  MaxIter:", Best_model._java_obj.parent().getMaxIter())

#Print "RegParam"
print("  RegParam:", Best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 20
  MaxIter: 10
  RegParam: 0.1


In [17]:
# Calculate the RMSE on test data using the best set of parameters obtained after cross validation
print("Best RMSE value is: ", evaluator.evaluate(Best_model.transform(test)))

Best RMSE value is:  0.8202128595787003


Checking Predictions

In [30]:
pred = Best_model.transform(test)

In [31]:
pred.show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|234926|    148|   4.0| 2.9755378|
| 50155|    148|   3.0|  2.947558|
|207939|    148|   3.0|  2.674797|
|220572|    148|   2.0| 2.7382836|
|244192|    148|   3.0| 2.4370253|
|108082|    148|   3.0| 2.7965746|
| 60382|    148|   4.0| 3.3882828|
| 28455|    148|   2.0|  2.379422|
|117490|    148|   3.5| 2.6895547|
|196553|    148|   2.5| 2.8994641|
+------+-------+------+----------+
only showing top 10 rows



In [32]:
pred.join(movies, "movieId").select("userId","title","genres","prediction").show(5)

+------+--------------------+----------+
|userId|               title|prediction|
+------+--------------------+----------+
|234926|Awfully Big Adven...| 2.9755378|
| 50155|Awfully Big Adven...|  2.947558|
|207939|Awfully Big Adven...|  2.674797|
|220572|Awfully Big Adven...| 2.7382836|
|244192|Awfully Big Adven...| 2.4370253|
+------+--------------------+----------+
only showing top 5 rows



In [70]:
for_an_user = pred.where(pred.userId==234926).join(movies, "movieId").join(links, "movieId").select("userId","title","tmdbId","genres","prediction")
for_an_user.show(5)

+------+--------------------+------+--------------------+----------+
|userId|               title|tmdbId|              genres|prediction|
+------+--------------------+------+--------------------+----------+
|234926|Awfully Big Adven...| 22279|               Drama| 2.9755378|
|234926|      Aladdin (1992)|   812|Adventure|Animati...| 3.2513719|
|234926|  French Kiss (1995)|   397|Action|Comedy|Rom...| 3.0609913|
|234926|Star Trek: Genera...|   193|Adventure|Drama|S...| 2.9618726|
|234926|  Client, The (1994)| 10731|Drama|Mystery|Thr...| 3.1212866|
+------+--------------------+------+--------------------+----------+
only showing top 5 rows



In [71]:
from IPython.display import Image
from IPython.display import display

In [72]:
import webbrowser
link = "https://www.themoviedb.org/movie"
for movie in for_an_user.take(2):
  url = link+str(movie.tmdbId)
  print(movie.title)
  webbrowser.open(url)

Awfully Big Adventure, An (1995)
Aladdin (1992)


Attempt 2

In [18]:
# Get top 10 movie recommendations for all users
movie_recommendation = Best_model.recommendForAllUsers(10)
movie_recommendation.show()
import pandas as pd
movie_recommendation = movie_recommendation.toPandas()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[[151989, 5.28400...|
|   463|[[177209, 3.45509...|
|   471|[[177209, 5.04457...|
|   496|[[177209, 5.62510...|
|   833|[[183947, 4.90726...|
|  1088|[[151989, 5.49119...|
|  1238|[[72032, 3.367489...|
|  1342|[[152986, 3.20153...|
|  1580|[[177209, 5.73612...|
|  1591|[[127252, 6.06560...|
|  1645|[[151989, 6.00942...|
|  1829|[[151989, 4.59136...|
|  1959|[[177209, 5.80286...|
|  2122|[[177209, 5.42111...|
|  2142|[[177209, 5.96326...|
|  2366|[[151989, 6.15353...|
|  2659|[[151989, 4.79692...|
|  2866|[[177209, 6.26189...|
|  3175|[[177209, 2.64555...|
|  3749|[[183947, 6.22231...|
+------+--------------------+
only showing top 20 rows



In [19]:
users = []
recommendations = []

# For the entire data iterate 
for i in range(len(movie_recommendation)):

  # First we add user_id to the list
  users.append(movie_recommendation.iloc[i,0])
  user_recs = "" 

  # Fetching only the item ID's from the recommendations
  for item in movie_recommendation.iloc[i,1]:
     user_recs = user_recs + ", " + str(item.asDict()["movieId"])

  # Append the itemID's to a list
  recommendations.append(user_recs[2:])

#Convert to dataframe
recommendations_df = pd.DataFrame(data = zip(users, recommendations), columns=["user", "MovieID"])

In [20]:
#Check the users and the top 10 movie recommendations for the first 10 users
recommendations_df.head(10)

,user,MovieID
0,148,"151989, 155923, 187873, 177209, 101706, 192089..."
1,463,"177209, 192089, 151989, 187873, 155923, 187125..."
2,471,"177209, 127252, 190707, 159761, 183947, 151989..."
3,496,"177209, 159017, 183947, 192089, 190707, 151989..."
4,833,"183947, 184299, 177209, 183845, 155923, 187873..."
5,1088,"151989, 122222, 177209, 187873, 155923, 138908..."
6,1238,"72032, 178147, 128667, 121919, 108050, 111762,..."
7,1342,"152986, 152035, 168090, 156396, 151989, 177879..."
8,1580,"177209, 151989, 157789, 127252, 183947, 190707..."
9,1591,"127252, 177209, 117352, 190707, 184299, 159761..."


In [21]:
#write to a text file
recommendations_df.to_csv('/content/recommendation_project.txt', sep='\t', index=False)